In [40]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.cluster import KMeans

Import del dataset in un DataFrame

In [41]:
data = pd.read_csv('Data/25day_dataset.csv')

In [42]:
label = []
for i in range(len(data)):
    if data['wahing_machine'].iloc[i]==0 and data['dishwasher'].iloc[i]==0 and data['oven'].iloc[i]==0:
        label.append(0)
    elif data['wahing_machine'].iloc[i]>0:
        label.append(1)
    elif data['dishwasher'].iloc[i]>0:
        label.append(2)
    elif data['oven'].iloc[i]>0:
        label.append(3)
data['Class'] = label

In [44]:
Test_set1 =data[data['DateTime'].str.startswith('2022-01-01')]
Test_set2 =data[data['DateTime'].str.startswith('2022-01-02')]
Test_set3 =data[data['DateTime'].str.startswith('2022-01-03')]
Test_set4 =data[data['DateTime'].str.startswith('2022-01-04')]
Test_set5 =data[data['DateTime'].str.startswith('2022-01-05')]
Test_set6 =data[data['DateTime'].str.startswith('2022-01-06')]
Test_set7 =data[data['DateTime'].str.startswith('2022-01-07')]
Test_set8 =data[(data['DateTime'].str.startswith('2022-01-08'))]
Test_set9 =data[(data['DateTime'].str.startswith('2022-01-10')) | (data['DateTime'].str.startswith('2022-01-11'))]
Test_set10 =data[(data['DateTime'].str.startswith('2022-01-12')) | (data['DateTime'].str.startswith('2022-01-13'))]
test_set_list= [Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6, Test_set7, Test_set8, Test_set9, Test_set10]

In [49]:
f_score={}
recall={}

for i in range(len(test_set_list)):
    
    print("start cleaning")
    dfi = data.drop(test_set_list[i].index)
    y_= dfi.Class
    X_= dfi.drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
    
    X = X_.groupby(np.arange(len(dfi))//5).mean()
    y = y_.groupby(np.arange(len(dfi))//5).agg(lambda x:x.value_counts().index[0])
    print("stop cleaning")

    
    two= int((y.values == 2).sum())
    three= int((y.values == 3).sum())
    undersample = RandomUnderSampler(sampling_strategy={0: 10000, 1: 10000, 2: two, 3: three})
    X_new, y_new = undersample.fit_resample(X, y)
    
    
    clf = RandomForestClassifier(
                                   class_weight= {0: 9, 1: 1, 2: 1, 3: 3},
                                   criterion= 'entropy', 
                                   max_depth= 10,
                                   max_features= 'sqrt',
                                   max_samples= 0.3,
                                   n_estimators= 50,
                                )
    clf.fit(X_new, y_new)
    
    X_test = test_set_list[i].copy().drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
    y_test = test_set_list[i].Class
    y_preds = clf.predict(X_test)
    
    print("CM:\n" + str(confusion_matrix(y_test,y_preds)) + "\n")
    print(classification_report(y_test,y_preds))
    r = recall_score(y_test,y_preds, average= None) 
    f = f1_score(y_test,y_preds, average= None)
    f_score[i] = f
    recall[i] = r 
    
f1= pd.DataFrame.from_dict(f_score, orient='index').replace(0, np.nan)
print(f1.mean(axis=0, skipna=True))
r1= pd.DataFrame.from_dict(recall, orient='index').replace(0, np.nan)
print(r1.mean(axis=0, skipna=True))

start cleaning
stop cleaning
CM:
[[78897   489  2173   106]
 [   76  3977     0     0]
 [    0     0     0     0]
 [  143     0     0   539]]

              precision    recall  f1-score   support

           0       1.00      0.97      0.98     81665
           1       0.89      0.98      0.93      4053
           2       0.00      0.00      0.00         0
           3       0.84      0.79      0.81       682

    accuracy                           0.97     86400
   macro avg       0.68      0.68      0.68     86400
weighted avg       0.99      0.97      0.98     86400

start cleaning


C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

stop cleaning
CM:
[[76244   888   465   672]
 [  566  6434     1     7]
 [    0     0     0     0]
 [   49     0     0  1074]]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     78269
           1       0.88      0.92      0.90      7008
           2       0.00      0.00      0.00         0
           3       0.61      0.96      0.75      1123

    accuracy                           0.97     86400
   macro avg       0.62      0.71      0.66     86400
weighted avg       0.98      0.97      0.97     86400

start cleaning


C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

stop cleaning
CM:
[[78816   549   324  1406]
 [  166   834     1    23]
 [  379   119  3078     1]
 [    0     0     0   704]]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     81095
           1       0.56      0.81      0.66      1024
           2       0.90      0.86      0.88      3577
           3       0.33      1.00      0.50       704

    accuracy                           0.97     86400
   macro avg       0.70      0.91      0.76     86400
weighted avg       0.98      0.97      0.97     86400

start cleaning
stop cleaning
CM:
[[81381   143   100   131]
 [    0     0     0     0]
 [    0     0     0     0]
 [  694  1056     3  2892]]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     81755
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.96      0.62      0.75      4645

    accuracy          

...

stop cleaning
CM:
[[78541   387  2062   449]
 [  148  4007     0     0]
 [    0     0     0     0]
 [  412     2     0   392]]

              precision    recall  f1-score   support

           0       0.99      0.96      0.98     81439
           1       0.91      0.96      0.94      4155
           2       0.00      0.00      0.00         0
           3       0.47      0.49      0.48       806

    accuracy                           0.96     86400
   macro avg       0.59      0.60      0.60     86400
weighted avg       0.98      0.96      0.97     86400

start cleaning


C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

stop cleaning
CM:
[[166561    496    555    619]
 [   111   2674      0      5]
 [     0      0      0      0]
 [   635      0      0   1144]]

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    168231
           1       0.84      0.96      0.90      2790
           2       0.00      0.00      0.00         0
           3       0.65      0.64      0.65      1779

    accuracy                           0.99    172800
   macro avg       0.62      0.65      0.63    172800
weighted avg       0.99      0.99      0.99    172800

start cleaning


C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

stop cleaning
CM:
[[156434    785   1867    836]
 [   393   6091      0      6]
 [   225      0   3238      0]
 [    87      1      1   2836]]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    159922
           1       0.89      0.94      0.91      6490
           2       0.63      0.94      0.76      3463
           3       0.77      0.97      0.86      2925

    accuracy                           0.98    172800
   macro avg       0.82      0.96      0.88    172800
weighted avg       0.98      0.98      0.98    172800

0    0.987094
1    0.878459
2    0.841537
3    0.693839
dtype: float64
0    0.979612
1    0.933133
2    0.887347
3    0.784575
dtype: float64


In [53]:
f1.mean(axis=0, skipna=True).mean()

0.8502324391605971

In [51]:
r1.mean(axis=0, skipna=True).mean()

0.896166876836433